# Introduction to CUDA Kernel Tiling and GPU Profiling

This notebook provides a gentle introduction to:
1. Understanding GPU memory hierarchy
2. The concept of tiling in CUDA
3. How to implement tiled matrix multiplication
4. Profiling CUDA kernels with NVIDIA Nsight Compute

## 1. Understanding GPU Memory Hierarchy

Before discussing tiling, it's important to understand the memory hierarchy in CUDA GPUs:

| Memory Type | Scope | Lifetime | Speed | Size |
|-------------|-------|----------|-------|-------|
| Global Memory | All threads | Application | Slowest | Largest (Several GB) |
| Shared Memory | Thread block | Block | Fast | Limited (KB per block) |
| Registers | Thread | Thread | Fastest | Very limited |

The goal of most optimizations is to minimize accesses to global memory (which is slow) and maximize the use of shared memory and registers (which are fast).

## 2. What is Tiling?

Tiling is an optimization technique that breaks a large computation into smaller chunks (tiles) that can fit in faster memory. In the context of CUDA:

1. **Load data from global memory to shared memory** in tiles
2. **Process the data within the tile** using threads in the same block
3. **Move to the next tile** until all data is processed

This approach significantly reduces global memory accesses, which are a major bottleneck in GPU computing.

### Visual Explanation of Tiling in Matrix Multiplication

For matrix multiplication C = A * B:

**Without tiling:**
- Each thread loads multiple elements from A and B from global memory
- Performs multiplications and additions
- Writes one element to C

**With tiling:**
- Threads in a block cooperatively load a tile of A and a tile of B into shared memory
- Each thread uses these shared memory tiles for calculations
- Move to the next tile until the entire row/column is processed
- Write result to C

```
Without Tiling:                  With Tiling:
+-------+                        +-------+
|       |                        |       |
| A     |                        | A     |
|       |                        |       |
+-------+                        +-------+
                                   |    |
+-------+     ==>     +-------+  +---+ +---+
|       |                        |Tile| |Tile|
| B     |                        +---+ +---+
|       |                          |     |
+-------+                        Shared Memory
                                     |
                                Fast Computation
```

## 3. Matrix Multiplication Without Tiling

Let's first look at a basic CUDA kernel for matrix multiplication without tiling:

In [ ]:
%%cuda
#include <stdio.h>

// Matrix multiplication kernel - without tiling
__global__ void matrixMulNaive(float* A, float* B, float* C, int N) {
    // Calculate row and column indices for this thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Check if indices are within the matrix dimensions
    if (row < N && col < N) {
        float sum = 0.0f;
        // Compute the dot product of the row of A and column of B
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

### Performance Issues with Naive Implementation

In this naive implementation:
- Each thread reads N elements from matrix A and N elements from matrix B from global memory
- For an N×N matrix multiplication, this results in 2×N×N×N global memory accesses
- Global memory accesses are slow and become a bottleneck for performance
- There is no data reuse between threads in the same block

Let's examine the memory access pattern more closely:

1. **Matrix A Access**:
   - Threads in the same row access consecutive elements in the same row of A
   - This is coalesced access, which is efficient

2. **Matrix B Access**:
   - Threads in the same column access elements from different rows in B
   - This is strided access (column-wise), which is inefficient
   - Each memory transaction fetches more data than needed, wasting bandwidth

## 4. Matrix Multiplication With Tiling

Now let's implement the tiled version, which uses shared memory to reduce global memory accesses:

In [ ]:
%%cuda
#include <stdio.h>

// Matrix multiplication kernel - with tiling
#define TILE_SIZE 16

__global__ void matrixMulTiled(float* A, float* B, float* C, int N) {
    // Shared memory for the tiles of A and B
    __shared__ float sharedA[TILE_SIZE][TILE_SIZE];
    __shared__ float sharedB[TILE_SIZE][TILE_SIZE];
    
    // Calculate row and column indices for this thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    float sum = 0.0f;
    
    // Loop over all tiles needed to compute the element
    for (int t = 0; t < (N + TILE_SIZE - 1) / TILE_SIZE; t++) {
        // Load the tiles from global memory to shared memory
        if (row < N && t * TILE_SIZE + threadIdx.x < N) {
            sharedA[threadIdx.y][threadIdx.x] = A[row * N + t * TILE_SIZE + threadIdx.x];
        } else {
            sharedA[threadIdx.y][threadIdx.x] = 0.0f;
        }
        
        if (col < N && t * TILE_SIZE + threadIdx.y < N) {
            sharedB[threadIdx.y][threadIdx.x] = B[(t * TILE_SIZE + threadIdx.y) * N + col];
        } else {
            sharedB[threadIdx.y][threadIdx.x] = 0.0f;
        }
        
        // Synchronize to make sure the tiles are loaded
        __syncthreads();
        
        // Perform computation using the tiles
        for (int k = 0; k < TILE_SIZE; k++) {
            sum += sharedA[threadIdx.y][k] * sharedB[k][threadIdx.x];
        }
        
        // Synchronize to make sure the computation is done
        __syncthreads();
    }
    
    // Write the result to global memory
    if (row < N && col < N) {
        C[row * N + col] = sum;
    }
}

### Key Aspects of the Tiled Implementation

1. **Shared Memory Allocation**
   - We allocate shared memory tiles for parts of matrices A and B
   - The tile size affects performance and should be tuned for specific GPUs

2. **Tile Loading**
   - Each thread loads one element of the tile
   - The loading pattern is designed to achieve coalesced memory access
   - Boundary checks ensure we don't access out-of-bounds memory

3. **Thread Synchronization**
   - `__syncthreads()` ensures all threads in a block have loaded their data before computation begins
   - This barrier is essential for correctness

4. **Computation**
   - Each thread computes the dot product using the shared memory tiles
   - Multiple tiles are processed to compute the final result
   - Shared memory access is much faster than global memory

5. **Memory Access Pattern**
   - The tiled approach significantly reduces global memory accesses
   - Each element in the tile is reused multiple times from fast shared memory
   - Both A and B are loaded in a coalesced pattern

## 5. Host Code for Testing the Kernels

Let's implement host code to test and compare both kernels:

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Kernel definitions from above would go here

// Utility function to check for CUDA errors
#define CHECK_CUDA_ERROR(call) do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
        exit(1); \
    } \
} while(0)

// Initialize a matrix with random values
void initMatrix(float* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = rand() / (float)RAND_MAX;
    }
}

// Main function
int main() {
    int N = 1024;  // Matrix size (N x N)
    size_t bytes = N * N * sizeof(float);
    
    // Allocate host memory
    float* h_A = (float*)malloc(bytes);
    float* h_B = (float*)malloc(bytes);
    float* h_C_naive = (float*)malloc(bytes);
    float* h_C_tiled = (float*)malloc(bytes);
    
    // Initialize input matrices
    initMatrix(h_A, N);
    initMatrix(h_B, N);
    
    // Allocate device memory
    float* d_A, *d_B, *d_C;
    CHECK_CUDA_ERROR(cudaMalloc(&d_A, bytes));
    CHECK_CUDA_ERROR(cudaMalloc(&d_B, bytes));
    CHECK_CUDA_ERROR(cudaMalloc(&d_C, bytes));
    
    // Copy data from host to device
    CHECK_CUDA_ERROR(cudaMemcpy(d_A, h_A, bytes, cudaMemcpyHostToDevice));
    CHECK_CUDA_ERROR(cudaMemcpy(d_B, h_B, bytes, cudaMemcpyHostToDevice));
    
    // Define grid and block dimensions
    dim3 blockDim(16, 16);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);
    
    // Create CUDA events for timing
    cudaEvent_t start, stop;
    CHECK_CUDA_ERROR(cudaEventCreate(&start));
    CHECK_CUDA_ERROR(cudaEventCreate(&stop));
    float milliseconds = 0;
    
    // Run naive kernel
    CHECK_CUDA_ERROR(cudaEventRecord(start));
    matrixMulNaive<<<gridDim, blockDim>>>(d_A, d_B, d_C, N);
    CHECK_CUDA_ERROR(cudaEventRecord(stop));
    CHECK_CUDA_ERROR(cudaEventSynchronize(stop));
    CHECK_CUDA_ERROR(cudaEventElapsedTime(&milliseconds, start, stop));
    printf("Naive kernel execution time: %.3f ms\n", milliseconds);
    
    // Copy result back to host
    CHECK_CUDA_ERROR(cudaMemcpy(h_C_naive, d_C, bytes, cudaMemcpyDeviceToHost));
    
    // Run tiled kernel
    CHECK_CUDA_ERROR(cudaEventRecord(start));
    matrixMulTiled<<<gridDim, blockDim>>>(d_A, d_B, d_C, N);
    CHECK_CUDA_ERROR(cudaEventRecord(stop));
    CHECK_CUDA_ERROR(cudaEventSynchronize(stop));
    CHECK_CUDA_ERROR(cudaEventElapsedTime(&milliseconds, start, stop));
    printf("Tiled kernel execution time: %.3f ms\n", milliseconds);
    
    // Copy result back to host
    CHECK_CUDA_ERROR(cudaMemcpy(h_C_tiled, d_C, bytes, cudaMemcpyDeviceToHost));
    
    // Verify results (simplified check)
    float maxError = 0.0f;
    for (int i = 0; i < N * N; i++) {
        maxError = max(maxError, abs(h_C_naive[i] - h_C_tiled[i]));
    }
    printf("Max error between naive and tiled implementations: %f\n", maxError);
    
    // Free device memory
    CHECK_CUDA_ERROR(cudaFree(d_A));
    CHECK_CUDA_ERROR(cudaFree(d_B));
    CHECK_CUDA_ERROR(cudaFree(d_C));
    
    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C_naive);
    free(h_C_tiled);
    
    // Destroy CUDA events
    CHECK_CUDA_ERROR(cudaEventDestroy(start));
    CHECK_CUDA_ERROR(cudaEventDestroy(stop));
    
    return 0;
}

## 6. Understanding Performance Benefits of Tiling

Let's analyze the theoretical performance improvement with tiling:

### Global Memory Access Analysis

**Without Tiling:**
- Each thread reads N elements from A and N elements from B
- For N×N threads, that's 2×N³ global memory accesses

**With Tiling (using TILE_SIZE×TILE_SIZE tiles):**
- Each block of TILE_SIZE×TILE_SIZE threads reads N/TILE_SIZE tiles
- Each tile requires 2×TILE_SIZE² global memory accesses
- Total global memory accesses: 2×N²×N/TILE_SIZE = 2×N³/TILE_SIZE

**Theoretical Speedup:**
- Ratio of global memory accesses: (2×N³) / (2×N³/TILE_SIZE) = TILE_SIZE
- With TILE_SIZE = 16, we expect up to 16× fewer global memory accesses

In practice, the speedup will be limited by other factors such as memory bandwidth, latency, thread synchronization overhead, and hardware occupancy.

## 7. Profiling with NVIDIA Nsight Compute

NVIDIA Nsight Compute is a powerful profiling tool for analyzing CUDA kernel performance. Here's how to use it to profile and analyze our matrix multiplication kernels:

### Basic Profiling Commands

```bash
# Compile with line information for source-level profiling
nvcc -o matrix_multiply matrix_multiply.cu --lineinfo

# Basic profiling
ncu ./matrix_multiply

# Save profiling results to a file
ncu -o profile_results ./matrix_multiply

# Comprehensive profiling with all metrics
ncu --set full ./matrix_multiply

# Profile specific kernel by name
ncu --kernel-name matrixMulTiled ./matrix_multiply

# Profile with specific metrics/sections
ncu --section SpeedOfLight ./matrix_multiply
```

### Profiling Process Step by Step

1. **Compile your code with debug information**
   - Use the `--lineinfo` flag with nvcc to include source information
   - This helps correlate performance metrics with specific lines of code

2. **Run basic profiling**
   - Start with `ncu ./your_application` to get an overview
   - This provides a summary of kernel launches and basic metrics

3. **Focus on specific kernels**
   - Use `--kernel-name` to profile just the kernels you're interested in
   - For multiple kernel launches, use `--kernel-id :::1` to profile only one instance

4. **Collect specific metrics**
   - Use built-in metric sets like `--set full` or `--section SpeedOfLight`
   - For targeted analysis, specify individual metrics

5. **Analyze results**
   - Use the Nsight Compute UI (`ncu-ui`) to visualize and analyze results
   - Compare different kernel implementations side by side

### Key Metrics to Analyze

When profiling our matrix multiplication kernels, pay attention to these key metrics:

1. **Memory-related metrics:**
   - Global Memory Load/Store Throughput
   - Shared Memory Load/Store Throughput
   - L1/L2 Cache Hit Rate
   - Memory Bandwidth Utilization

2. **Compute-related metrics:**
   - SM (Streaming Multiprocessor) Utilization
   - Warp Execution Efficiency
   - Achieved Occupancy
   - Instructions Per Cycle (IPC)

3. **Limiting factors:**
   - Compute Bound vs Memory Bound
   - Stall Reasons (Memory, Execution, Synchronization)
   - Memory Dependency
   - Execution Dependency

### Example Profiling Output

Here's an example of what profiling output might look like for our kernels:

```
==== Naive Matrix Multiplication Kernel ====
Memory Throughput: 80 GB/s (30% of peak)
Compute Throughput: 600 GFLOPS (20% of peak)
Primary Bottleneck: Memory Bandwidth
Warp Stall Reasons:
  - Memory Dependency: 70%
  - Execution Dependency: 20%
  - Other: 10%

==== Tiled Matrix Multiplication Kernel ====
Memory Throughput: 30 GB/s (11% of peak)
Compute Throughput: 1800 GFLOPS (60% of peak)
Primary Bottleneck: Compute
Warp Stall Reasons:
  - Execution Dependency: 60%
  - Memory Dependency: 25%
  - Synchronization: 10%
  - Other: 5%
```

This example illustrates the shift from memory-bound to compute-bound execution when using tiling, which is a key indicator of successful memory optimization.

## 8. Understanding Profiling Results

When analyzing profiling results, look for these patterns:

1. **Memory-bound kernels:**
   - High percentage of memory-related stalls
   - Low arithmetic intensity (few computations per memory access)
   - Low IPC (Instructions Per Cycle)
   - Optimizations should focus on reducing memory accesses

2. **Compute-bound kernels:**
   - High percentage of execution-related stalls
   - High arithmetic intensity
   - Higher IPC
   - Optimizations should focus on improving instruction-level parallelism

3. **Synchronization-bound kernels:**
   - High percentage of synchronization stalls
   - Many `__syncthreads()` calls
   - Optimizations should focus on reducing synchronization points

The naive matrix multiplication is typically memory-bound, while the tiled version may become compute-bound, depending on the tile size and hardware capabilities.

## 9. Optimization Strategies Beyond Basic Tiling

While basic tiling provides significant performance improvements, there are several advanced techniques to further optimize our kernel:

1. **Prefetching:**
   - Load the next tile while computing on the current tile
   - Overlaps memory transfers with computation
   - Can reduce memory stalls

2. **Loop Unrolling:**
   - Explicitly unroll the inner computation loop
   - Reduces loop overhead and allows for better instruction-level parallelism
   - Example:
     ```c
     #pragma unroll
     for (int k = 0; k < TILE_SIZE; k++) {
         sum += sharedA[threadIdx.y][k] * sharedB[k][threadIdx.x];
     }
     ```

3. **Register Blocking/Thread Coarsening:**
   - Have each thread compute multiple output elements
   - Reduces shared memory accesses and synchronization points
   - Increases register pressure but can improve performance

4. **Vectorized Memory Access:**
   - Use vector types (float4, etc.) for coalesced memory access
   - Can significantly improve memory throughput
   - Especially beneficial for older GPU architectures

5. **Bank Conflict Avoidance:**
   - Pad shared memory to avoid bank conflicts
   - Example: `__shared__ float sharedA[TILE_SIZE][TILE_SIZE+1];`
   - Especially important for matrix operations with strided access patterns

## 10. Exercise: Experiment with Tile Sizes

As an exercise, you can modify the code to experiment with different tile sizes and observe their impact on performance.

Try changing the TILE_SIZE value in the code above to 8, 16, 32, or 64, and compare the execution times.

Questions to consider:
1. How does the tile size affect performance?
2. Why might very small or very large tile sizes perform poorly?
3. How do the optimal tile sizes relate to the hardware specifications of your GPU (warp size, shared memory size, etc.)?

For example, you could modify the code to look like this:

```c
// Try different tile sizes
#define TILE_SIZE_8 8
#define TILE_SIZE_16 16
#define TILE_SIZE_32 32
#define TILE_SIZE_64 64

// Create four different kernel functions with different tile sizes
// ...

// Then time each one and compare the results
// ...
```

## 11. Common Pitfalls and Debugging Tips

When implementing tiled CUDA kernels, watch out for these common issues:

1. **Incorrect Synchronization**
   - Missing `__syncthreads()` can lead to race conditions
   - Placing them inside conditional blocks can cause deadlocks

2. **Out-of-Bounds Memory Access**
   - Always check array boundaries, especially for non-square matrices or tile sizes that don't evenly divide the matrix dimensions
   - Use proper padding or boundary checks

3. **Bank Conflicts**
   - Shared memory is organized into banks, and simultaneous access to the same bank by different threads in a warp causes serialization
   - Use padding to avoid bank conflicts

4. **Occupancy Issues**
   - Using too much shared memory or registers per thread can reduce occupancy
   - Use the occupancy calculator or NVIDIA Nsight Compute to analyze occupancy

5. **Debugging Tips**
   - Start with small matrices to verify correctness
   - Use `printf` in your kernel for debugging (only in small test cases)
   - Use CUDA error checking macros
   - Profile and optimize one step at a time

## 12. Conclusion

In this notebook, we've explored the concept of tiling as a fundamental optimization technique for GPU programming. Key takeaways include:

1. **Memory hierarchy awareness** is crucial for GPU performance optimization
2. **Tiling** reduces global memory accesses by leveraging faster shared memory
3. **Thread synchronization** is essential for correctness in tiled implementations
4. **Performance profiling** helps identify bottlenecks and guide optimizations
5. **Advanced optimization techniques** can further improve performance beyond basic tiling

By understanding and applying these concepts, you can significantly improve the performance of your CUDA kernels, especially for memory-bound applications like matrix multiplication.

Tiling is just one of many optimization techniques in the GPU programmer's toolkit, but it's one of the most fundamental and widely applicable approaches for improving memory performance.

## References

1. NVIDIA CUDA Programming Guide: https://docs.nvidia.com/cuda/cuda-c-programming-guide/
2. NVIDIA Nsight Compute Documentation: https://docs.nvidia.com/nsight-compute/
3. CUDA Best Practices Guide: https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/
4. Matrix-Matrix Multiplication on the GPU: https://www.quantstart.com/articles/Matrix-Matrix-Multiplication-on-the-GPU-with-Nvidia-CUDA/
5. How to Optimize a CUDA Matmul Kernel: https://siboehm.com/articles/22/CUDA-MMM